# In this notebook the used threshold will be discussed and tested

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt
from pathlib import Path
import os
from pipeline import undistort_image

In [2]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillConvexPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [3]:
# Make a list of test images
images = glob.glob('../test_images/*.jpg')
output_dir = "../output_images/threshold/"
Path(output_dir).mkdir(parents=True, exist_ok=True)

# Parameters to experiment #################
s_thresh=(150, 255)
sx_thresh=(23, 150)
############################################

for filename in images:
    img = cv2.imread(filename)

    # Undistort image with previously computes parameters
    undst = undistort_image(img)

    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(undst, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]

    # Apply sobel x to l chanel
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1

    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()

    ax1.imshow(mpimg.imread(filename))
    ax1.set_title('Original Image', fontsize=40)

    ax2.imshow(color_binary)
    ax2.set_title('Pipeline Result', fontsize=40)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

    f.savefig(output_dir + os.path.basename(filename))
    f.show()

## Threshold conclusions:

The images were succesfully transformed to a threshold image which indicates the position of the lanes. To make this image there are 2 contributions the x sobel derivate apply to the l chanel marked in green in the images, and the s chanel simply filtered with a threshold markes in blue. The obtained images can be found in output_images/threshold. For example:

![Lanes Image](../output_images/threshold/test2.jpg) 

![Lanes Image](../output_images/threshold/test3.jpg) 

With this experiment the function threshold was implemented in pipeline.py

In [8]:
# Testing threshold function
from pipeline import threshold

# Make a list of test images
images = glob.glob('../test_images/*.jpg')

for filename in images:
    img = cv2.imread(filename)

    # Undistort image with previously computes parameters
    undst = undistort_image(img)
    
    # Obtain bitmap
    color_binary = threshold(undst)

    # Stack each channel
    color_binary_to_plot = np.dstack(( color_binary, color_binary, color_binary)) * 255

    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()

    ax1.imshow(mpimg.imread(filename))
    ax1.set_title('Original Image', fontsize=40)

    ax2.imshow(color_binary_to_plot)
    ax2.set_title('Pipeline Result', fontsize=40)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

    f.show()